# Parameter Exploration

In [ ]:
import random
from timeit import default_timer as timer

import networkx as nx

import matplotlib.pyplot as plt

from community_detection.utils import Partition, argmax
from community_detection.leiden import leiden as leiden_o
from community_detection.louvain import louvain as louvain_o
from community_detection.quality_functions import CPM, Modularity

SEED = 0

In [ ]:
import datasets.cosnology as cosnology
G = cosnology.get_graph()
print(f"Loaded {len(G.nodes)=} and {len(G.edges)=}.")

Loaded len(G.nodes)=4039 and len(G.edges)=88234.


In [3]:

def leiden(*args, **kwargs):
    random.seed(SEED)
    start = timer()
    result = leiden_o(*args, **kwargs)
    runtime = timer() - start
    return result, runtime


def louvain(*args, **kwargs):
    random.seed(SEED)
    start = timer()
    result = louvain_o(*args, **kwargs)
    runtime = timer() - start
    return result, runtime

In [4]:
resolutions_mod = [0.40 + 0.02 * d for d in range(50)] # 0.40 ... 1.60
resolutions_cpm = [0.04 + 0.02 * d for d in range(50)] # 0.04 ... 0.98

In [5]:
from tqdm import tqdm

# Run louvain and leiden algorithms for all resolutions with the Modularity quality function
coms_louvain_mod, times_louvain_mod = map(list, zip(*[louvain(G, Modularity(res)) for res in tqdm(resolutions_mod, desc="Louvain Modularity")]))
coms_leiden_mod,  times_leiden_mod  = map(list, zip(*[leiden (G, Modularity(res)) for res in tqdm(resolutions_mod, desc="Leiden Modularity")]))
# Now repeat this with CPM as the quality function
coms_louvain_cpm, times_louvain_cpm = map(list, zip(*[louvain(G, CPM(res)) for res in tqdm(resolutions_cpm, desc="Louvain CPM")]))
coms_leiden_cpm,  times_leiden_cpm  = map(list, zip(*[leiden (G, CPM(res), θ=0.8) for res in tqdm(resolutions_cpm, desc="Leiden CPM")]))

Leiden CPM:   0%|          | 0/50 [00:00<?, ?it/s]

OverflowError: math range error

In [ ]:
mod = Modularity(1.0)
mod_louvain_mod = [mod(part) for part in coms_louvain_mod]
mod_leiden_mod  = [mod(part) for part in coms_leiden_mod]
mod_louvain_cpm = [mod(part) for part in coms_louvain_cpm]
mod_leiden_cpm  = [mod(part) for part in coms_leiden_cpm]

In [ ]:
idx_louvain_mod = argmax(lambda x: x, mod_louvain_mod)[2]
idx_leiden_mod  = argmax(lambda x: x, mod_leiden_mod)[2]
idx_louvain_cpm = argmax(lambda x: x, mod_louvain_cpm)[2]
idx_leiden_cpm  = argmax(lambda x: x, mod_leiden_cpm)[2]

print(f"Best result for Louvain algorithm with Mod.: Modularity={mod_louvain_mod[idx_louvain_mod]:0.5f} " +
      f"with γ={resolutions_mod[idx_louvain_mod]:.3f}, yielding {len(coms_louvain_mod[idx_louvain_mod])} communities")
print(f"Best result for Leiden algorithm with Mod.:  Modularity={mod_leiden_mod[idx_leiden_mod]:0.5f} " +
      f"with γ={resolutions_mod[idx_leiden_mod]:.3f}, yielding {len(coms_leiden_mod[idx_leiden_mod])} communities")

print(f"Best result for Louvain algorithm with CPM:  Modularity={mod_louvain_cpm[idx_louvain_cpm]:0.5f} " +
      f"with γ={resolutions_cpm[idx_louvain_cpm]:.3f}, yielding {len(coms_louvain_cpm[idx_louvain_cpm])} communities")
print(f"Best result for Leiden algorithm with CPM:   Modularity={mod_leiden_cpm[idx_leiden_cpm]:0.5f} " +
      f"with γ={resolutions_cpm[idx_leiden_cpm]:.3f}, yielding {len(coms_leiden_cpm[idx_leiden_cpm])} communities")

In [ ]:
# Set up two graphs, side by side, to plot the results in
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
ax2.yaxis.set_tick_params(labelbottom=True)

## Left graph, for modularity
ax1.set_title("Modularity")
ax1.set_xlabel('Resolution $γ$ of quality function')
ax1.set_ylabel('Quality of resulting partition')

## Right graph, for CPM
ax2.set_title("CPM")
ax2.set_xlabel('Resolution $γ$ of quality function')
ax2.set_ylabel('Quality of resulting partition')

# Draw the results for the modularity quality function
## Plot the modularity determined
ln_mod_louvain_mod, = ax1.plot(resolutions_mod, mod_louvain_mod, label='Quality Louvain')
ln_mod_leiden_mod,  = ax1.plot(resolutions_mod, mod_leiden_mod,  label='Quality Leiden')

## Draw the x marks where maximum quality is attained
ax1.set_prop_cycle(None)
ax1.plot(resolutions_mod[idx_louvain_mod], mod_louvain_mod[idx_louvain_mod], 'x')
ax1.plot(resolutions_mod[idx_leiden_mod],  mod_leiden_mod[idx_leiden_mod],   'x')

# Draw the results for the CPM quality function
## Plot the modularity determined
ln_mod_louvain_cpm, = ax2.plot(resolutions_cpm, mod_louvain_cpm, label='Louvain CPM')
ln_mod_leiden_cpm,  = ax2.plot(resolutions_cpm, mod_leiden_cpm,  label='Leiden CPM')

## Draw the x marks where maximum quality is attained
ax2.set_prop_cycle(None)
ax2.plot(resolutions_cpm[idx_louvain_cpm], mod_louvain_cpm[idx_louvain_cpm], 'x')
ax2.plot(resolutions_cpm[idx_leiden_cpm],  mod_leiden_cpm[idx_leiden_cpm],   'x')

# Put a legend there
lgd = fig.legend(handles=[ln_mod_louvain_mod, ln_mod_leiden_mod], ncol=4,
                 loc='upper center', bbox_to_anchor=(0.5, 0.0))

# Draw the plot
fig.tight_layout()
fig.subplots_adjust(wspace=0.4)
plt.show()
# fig.savefig("quality_comparison.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# Set up two graphs, side by side, to plot the results in
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
ax2.yaxis.set_tick_params(labelbottom=True)

## Left graph, for modularity
ax1.set_title("Modularity")
ax1.set_xlabel('Resolution $γ$ of quality function')
ax1.set_ylabel('Number of detected communities')

## Plot the number of found communities
ln_cnt_louvain_mod, = ax1.plot(resolutions_mod, list(map(len, coms_louvain_mod)), label='Community count Louvain')
ln_cnt_leiden_mod,  = ax1.plot(resolutions_mod, list(map(len, coms_leiden_mod)), label='Community count Leiden')



## Right graph, for CPM
ax2.set_title("CPM")
ax2.set_xlabel('Resolution $γ$ of quality function')
ax2.set_ylabel('Number of detected communities')

## Plot the number of found communities
ln_cnt_louvain_cpm, = ax2.plot(resolutions_cpm, list(map(len, coms_louvain_cpm)), label='Community count')
ln_cnt_leiden_cpm,  = ax2.plot(resolutions_cpm, list(map(len, coms_leiden_cpm)), label='Community count')


# Put a legend there
lgd = fig.legend(handles=[ln_cnt_louvain_mod, ln_cnt_leiden_mod], ncol=4,
                 loc='upper center', bbox_to_anchor=(0.5, 0.0))

# Draw the plot
fig.tight_layout()
fig.subplots_adjust(wspace=0.4)
plt.show()
# fig.savefig("community_count.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# Export the graph, taking care to export the legend as well
# fig.savefig('figures/mod-cpm-comparison.pdf', bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# Set up two graphs, side by side, to plot the results in
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4), sharey=True)
ax2.yaxis.set_tick_params(labelbottom=True)

## Left graph, for modularity
ax1.set_title("Modularity")
ax1.set_xlabel('Resolution $γ$ of quality function')
ax1.set_ylabel('Runtime [s]')

## Right graph, for CPM
ax2.set_title("CPM")
ax2.set_xlabel('Resolution $γ$ of quality function')
ax2.set_ylabel('Runtime [s]')

# Draw the results for the modularity quality function
## Plot the modularity determined
ln_time_louvain_mod, = ax1.plot(resolutions_mod, times_louvain_mod, label='Runtime [s] Louvain')
ln_time_leiden_mod,  = ax1.plot(resolutions_mod, times_leiden_mod,  label='Runtime [s] Leiden')

# Draw the results for the CPM quality function
## Plot the modularity determined
ln_time_louvain_cpm, = ax2.plot(resolutions_cpm, times_louvain_cpm, label='Runtime [s] Louvain')
ln_time_leiden_cpm,  = ax2.plot(resolutions_cpm, times_leiden_cpm,  label='Runtime [s] Leiden')

# Put a legend there
lgd = fig.legend(handles=[ln_time_louvain_mod, ln_time_leiden_mod,], ncol=2,
                 loc='upper center', bbox_to_anchor=(0.5, 0.0))

# Draw the plot
fig.tight_layout()
fig.subplots_adjust(wspace=0.4)
plt.show()

In [ ]:
# Export the graph, taking care to export the legend as well
# fig.savefig('figures/mod-cpm-times.pdf', bbox_extra_artists=(lgd,), bbox_inches='tight')